# Home Energy Consumption Data

This example shows how to retrieve time series data on home energy consumption via the API and display them as charts on map popups.

## Initial setup

Use `CreateHomeEnergyMonitorDataset.ipynb` to generate the JSON files used to populate the database. Note that the notebook generates a separate measures file for each sensors, since a single file could lead to memory problems.

```
docker cp your_json_dir docker_web_1:/json
cd docker
docker-compose exec web bash
flask db init
flask db load /json/sensor_types.json
flask db load /json/sensors.json
find /json -name 'measures*.json' | while read f; do flask db load ${f}; done
```

In [1]:
import altair
import folium
import json
import pandas
import requests

from datetime import datetime, timedelta

In [2]:
FMT = "%Y-%m-%dT%H:%M:%SZ"

def avg(seq):
    return sum(seq) / len(seq)

## Get all sensors and display them on the map

In [3]:
sensors = requests.get('http://localhost:8000/api/v0.0/sensors').json()
print(f"n. sensors: {len(sensors)}")

n. sensors: 24


In [4]:
coords = [_["geometry"]["coordinates"] for _ in sensors]
locations = [[lat, lon] for lon, lat in coords]
map_center = [avg([_[0] for _ in locations]), avg([_[1] for _ in locations])]
m = folium.Map(location=map_center, zoom_start=13)
for loc in locations:
    folium.Marker(loc).add_to(m)
m

## Select sensors in a given area and time range

In [5]:
args = {
    'footprint': f'circle({tuple(map_center[::-1])}, 1000)',
    'after': '2017-05-01T00:00:00Z',
    'before':'2017-05-31T00:00:00Z'
}
sensors = requests.get('http://localhost:8000/api/v0.0/sensors', params=args).json()
print(f"n. selected sensors: {len(sensors)}")

n. selected sensors: 16


In [6]:
m = folium.Map(location=map_center, zoom_start=15)
for s in sensors:
    folium.Marker(s["geometry"]["coordinates"][::-1]).add_to(m)
m

## Add energy consumption chart to map popups

In [7]:
def energy_consumption_chart(sensor_code, after, before):
    params = {
        "after": after,
        "before": before,
        'bucket': 3600, # secs
        'op': 'sum'
    }
    res = requests.get(
        f"http://localhost:8000/api/v0.0/sensors/{sensor_code}/timeseries",
        params=params
    ).json()
    after_dt = datetime.strptime(after, FMT)
    datetimes = [after_dt + timedelta(seconds=_) for _ in res["timedelta"]]
    df = pandas.DataFrame({
        "datetime": datetimes,
        "energy": res["data"]
    })
    return altair.Chart(df).mark_line().encode(x="datetime", y="energy")

In [8]:
m = folium.Map(location=map_center, zoom_start=15)
for s in sensors:
    chart = energy_consumption_chart(s["code"], after=args["after"], before=args["before"])
    folium.Marker(
        location=s["geometry"]["coordinates"][::-1],
        popup=folium.Popup(max_width=450).add_child(folium.VegaLite(chart))
    ).add_to(m)
m